In [ ]:
%pip install langchain
%pip install -qU langchain-google-vertexai
%pip install -qU langchain-openai

In [7]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

from pydantic import BaseModel, Field

from typing import List

import json

In [8]:
import os

def read_file_from_path(path):
    with open(os.path.expanduser(path), 'r', encoding='utf-8') as file:
        return file.read().strip()

In [9]:
import os

os.environ["OPENAI_API_KEY"]=read_file_from_path('~/api_keys/OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"]=read_file_from_path('~/api_keys/GOOGLE_API_KEY')

# Critique Chain

User enters a question, LLM gets a response, and another LLM critiques the response

## Public

In [15]:
base_prompt = ChatPromptTemplate.from_template(
    'You are a researcher trying to answer the question "{question}". Make sure that you include specific '
    'examples or details backing up any generalizations')
base_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

critic_prompt = ChatPromptTemplate.from_template("A user asked <Question>{question}</Question> and received the response <Response>{response}</Response> from an LLM. Critique the quality of this response.")
critic_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

critique_chain = (
    RunnableParallel(question=RunnablePassthrough(), response=(base_prompt | base_model | StrOutputParser() | RunnableLambda(lambda s: s.strip())))
    | RunnablePassthrough.assign(critic_prompt=critic_prompt)
    | RunnablePassthrough.assign(critique=(RunnableLambda(itemgetter('critic_prompt')) | critic_model | StrOutputParser()))
)

## Testing

In [18]:
def output(d):
    divider = f"\n\n---------------------------------------------------------\n\n"
    format = lambda header, text: f"{header}\n\n{text}"
    return divider.join([format('USER', d['question']), format('LLM', d['response']), format('CRITIC', d['critique'])])

In [138]:
### Commented to allow running all cells. Uncomment to test ###

# critique_chain_io = critique_chain | RunnableLambda(output)
# question = input()
# result = critique_chain_io.invoke(question)
# print(result)

 did thomas jefferson support the united irishmen?


USER

did thomas jefferson support the united irishmen?

---------------------------------------------------------

LLM

The question of whether Thomas Jefferson supported the United Irishmen is complex and doesn't lend itself to a simple yes or no answer. While he expressed sympathy for their cause and engaged in actions that could be interpreted as tacit support, direct evidence of overt, active endorsement is lacking.  A nuanced analysis requires examining his actions and writings within the context of his political philosophy and the geopolitical realities of the time.

**Arguments for Jeffersonian Sympathy:**

* **Shared Republican Ideals:**  Jefferson was a staunch republican, deeply committed to principles of self-government, liberty, and resistance to tyranny.  The United Irishmen, seeking to overthrow British rule in Ireland and establish an independent republic, shared these ideals.  His writings frequently championed the rights of oppressed peoples, aligning with the United 

# Broad Claims

Takes a text block representing some kind of argument and finds claims that need concrete examples to stand up to criticism.

## Public

In [32]:
argument_schema = {
    "title": "argument_breakdown",
    "description": "A breakdown of the signifiant claims made in a line of reasoning in an academic argument",
    "type": "object",
    "properties": {
        "broad_claims": {
            "description": (
                "A list of claims that are vital to the main argument but are too broad to verify or disprove based on the evidence in the argument."
                "For the argument to be stronger, they would need to be broken down into more specific illustrative examples."
            ),
            "type": "array",
            "items": {
                "description": "An overly broad claim with guidance on how to make it modify to be more verifiable.",
                "type": "object",
                "properties": {
                    "excerpt": {
                        "descritpion": (
                            "The raw text of the claim. It should be copied verbatim aside from minimal standard quotation clarifications (e.g. bracketed"
                            "clarifications, ellipses in place of irrelevant content) needed for disambiguation when the quote is read out of context. They"
                            "should be short: no more than one or two sentences."),
                        "type": "string",
                    },
                    "examples_needed": {
                        "descritpion": "One or more aspect(s) of the claim for which concrete examples would strengthen the argument.",
                        "type": "array",
                        "items": {
                            "type": "string",
                        },
                    },
                },
                "required": ["excerpt", "examples_needed"],
            },
        },
    },
    "required": ["broad_claims"],
}

In [34]:
argument_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an academic journal editor trying to analyze the following argument from a journal submission."
               "Your job is only to break down the argument, not to evaluate its merits."),
    ("human", "{input}")
])
argument_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash").with_structured_output(argument_schema)

argument_chain = argument_prompt | argument_model

### Testing

In [36]:
### Commented to allow running all cells. Uncomment to test ###

# argument_breakdown = argument_chain.invoke({"input": """The question of whether Thomas Jefferson supported the United Irishmen is complex and doesn't lend itself to a simple yes or no answer. While he expressed sympathy for their cause and engaged in actions that could be interpreted as tacit support, direct evidence of overt, active endorsement is lacking.  A nuanced analysis requires examining his actions and writings within the context of his political philosophy and the geopolitical realities of the time.

# **Arguments for Jeffersonian Sympathy:**

# * **Shared Republican Ideals:**  Jefferson was a staunch republican, deeply committed to principles of self-government, liberty, and resistance to tyranny.  The United Irishmen, seeking to overthrow British rule in Ireland and establish an independent republic, shared these ideals.  His writings frequently championed the rights of oppressed peoples, aligning with the United Irishmen's grievances against British oppression.  This ideological affinity suggests a degree of sympathy, even if not necessarily active support.

# * **Diplomatic Maneuvers:**  As Secretary of State and later President, Jefferson engaged in diplomatic actions that indirectly benefited the United Irishmen.  While he couldn't openly support a rebellion against a major European power (Britain) without jeopardizing US neutrality and interests, certain policies could be interpreted as beneficial to the Irish cause.  For example, maintaining a strong, anti-British stance in foreign policy created a climate where Irish rebellion might find more tolerance internationally. This was not direct support, but a form of indirect aid by undermining the British.

# * **Correspondence and Associations:**  While direct evidence of Jefferson actively supporting the United Irishmen's rebellion is limited, his correspondence might reveal indirect support.  Historians would need to carefully examine his letters for any allusions to the movement or expressions of solidarity with its members.  However, given the secrecy surrounding the United Irishmen's activities, overt support in written correspondence would have been risky.  His association with individuals known to have supported the United Irishmen, while not necessarily proof of his own support, needs to be considered in the context of his broader network."""})

# print(json.dumps(argument_breakdown, indent=2))

Key 'parameters' is not supported in schema, ignoring


[
  {
    "args": {
      "broad_claims": [
        {
          "examples_needed": [
            "Specific examples of Jefferson's writings that champion the rights of oppressed peoples and how they align with the United Irishmen's grievances.",
            "Specific diplomatic actions taken by Jefferson that could be interpreted as indirectly benefiting the United Irishmen and a detailed explanation of how they did so.",
            "Specific examples of Jefferson's correspondence or associations that might reveal indirect support for the United Irishmen, including names of individuals and details of the correspondence or associations"
          ],
          "excerpt": "The question of whether Thomas Jefferson supported the United Irishmen is complex and doesn't lend itself to a simple yes or no answer. While he expressed sympathy for their cause and engaged in actions that could be interpreted as tacit support, direct evidence of overt, active endorsement is lacking.  A nuanced analy

# In Progress

In [22]:
argument_few_shot = [
    HumanMessage(
        (
            "The causes of World War I were multifaceted, involving a complex web of political, territorial, and economic factors. Some important factors included"
            "mutual defense alliances between major European powers, imperialistic rivalries over colonies and resources, militarism and the arms race, nationalism"
            "leading to heightened tensions. The assassination of Archduke Franz Ferdinand of Austria-Hungary acted as the immediate catalyst. These"
            "factors collectively created a volatile environment that ultimately led to the outbreak of war in 1914.",
        ),
        name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "argument_breakdown",
                "args": {
                    "broad_claims": [
                        {
                            "excerpt": "Some important factors included mutual defense alliances between major European powers, imperialistic rivalries over"
                            "colonies and resources, militarism and the arms race, nationalism leading to heightened tensions.",
                            "examples_needed": "TODO",
                        },
                        {
                            "excerpt": "The assassination of Archduke Franz Ferdinand of Austria-Hungary acted as the immediate catalyst.",
                            "examples_needed": "TODO",
                        }
                    ],
                },
                "id": "1",
            },
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="1"),
    # Some models also expect an AIMessage to follow any ToolMessages, so I may need to add an AIMessage here.
    # TODO: Repeat
]